# Les Optimisations de mix de produits ( produits mix )

## Introduction.


Les optimisations de type mix de produits sont les premières abordées en optimisation.

Elles permettent d'optimiser la production et le rendement commercial d'une usine ou d'une organisation agricole, et d'éviter en premier ressort des logiques de production aberrantes.

Elles permettent à une usine de savoir quels produits il vaut mieux produire dans son éventail de produits, afin de maximiser son profit.


Un exemple d'optimisation mixe de produits pour une organisation agricole 
dans mon application www.solvgraph.com :
<div style="text-align:center">
<img src="img/product-mix1.png">
</div>

Dans ses versions non linéaires, elles permettent plus de sophistication et permettent de coller à la réalité.

Dans cette étude, je vais utiliser plusieurs solveurs, et faire des optimisations linéaires et non 
linéaires.

Un exemple d'optimisation dans mon application www.solvgraph.com :
<div style="text-align:center">
<img src="img/product_mixe2.jpg">
</div>


Etude globale proposée par <b>Estelle Derrien - Github15000</b>

CREATION EN COURS


# Sommaire 

- 1. Maximisation de profit linéaire - Production de 3 objets.
        - Modélisation mathématique
        - Exemple
        - Résolution avec Python Pulp
            - Sans Boucle For
            - Avec Boucle For
        - Résolution avec Cplex
        - Résolution avec Excel
        - La minimisation de coûts ( Inverse..)
- 3. A quoi sert l'analyse de sensitivité ?
- 4. Ajout des Coûts variables.
- 5. Ajout de contraintes binaires.
- 6. Ajout de Multiples de.
- 7. Prise en compte des Invendus.
- 8. Ajout de contraintes Piecewize.
    - Description
    - Résolution avec Cplex
    - Résolution avec Python Pulp
- 9. Maximisation de profit non linéaire - Production de 3 objets.
    - Modélisation mathématique
    - Exemple
    - Exemple avec boucles For
    - Résolution avec Python Gekko
        - Sans Boucle For
        - Avec Boucle For
    - Résolution avec Cplex
    - Ajout de contraintes binaires
    - Ajout de contraintes piecewize


